In [1]:
# create a Fuel stream which takes pairs of (source, target)
# and converts them into (len(source_sentence) + 1) triples of (source, prefix, completion),
#      at training time, take the (source, prefix) inputs, and sample N
# system should also be able to propose just the 'EOS </S>' token -- if the target prefix looks like 
# the correct hypothesis, do nothing

# include direct machine translation as the '0' output -- given the <S> token,
# the system first presents a full hypothesis

# TODO: make sure <S> and </S> are correctly handled in beam search and evaluation

# WORKING: new evaluation metric, implemented as 
# `from machine_translation.evaluation import sentence_level_bleu`

# to evaluate the model on new data, we need three parallel files (sources, prefixes, reference_completions)
# when writing the output of the model, we output N completions per input, and separate the N outputs for each
# input by a blank line -- similar to WMT QE feature format

# for each line, compute the score, then average all of them to get the score for the dataset

# notes on baselines
# we should evaluate both F1 as in Ueffing and Ney, and NCDG for ranking
#     - plain F1 is the primary metric, since this fits a wider range of usecases
# the easiest baseline is to ignore the target prefix completely, and just generate a full hypothesis for the source
# - if the target prefix matches the prefix of the hypothesis, compute f1 as usual

In [2]:
import os

import logging
logging.basicConfig()
logger = logging.getLogger(__name__)

In [3]:
logger.setLevel(logging.DEBUG)

In [4]:
def imt_f1(hyp, ref):
    """
    compute Ueffing and Ney F1 for IMT
    
    Note that this function is agnostic about its inputs, as long as they
    are sequences. Thus the metric can be computed for sequences of characters, 
    words, phrases, etc...
    
    """
    
    # if both are empty, this is a perfect match
    if len(hyp) == 0 and len(ref) == 0:
        return 1.
    
    match_len = float(0)
    hyp_len = float(len(hyp))
    ref_len = float(len(ref))
    for h_sym, r_sym in zip(hyp, ref):
        if h_sym == r_sym:
            match_len += 1.
        else:
            break
    
    if match_len == 0:
        return 0.
    
    # ratio of characters in the prediction which are correct (low if prefix is too long)
    precision = match_len / hyp_len        
        
    # ratio of coverage of the reference (low if prefix is too short)
    recall = match_len / ref_len
    return 2 * ((precision * recall) / (precision + recall))

In [5]:
imt_f1('a sand', 'a sandwich made of gold')
# imt_f1('fish sandwich', 'a sandwich')

imt_f1('', '')
imt_f1('a sandw', 'a sandwi')

0.9333333333333333

In [6]:
def map_pair_to_imt_triples(source, reference, bos_token=None, eos_token=None):
    """
    Map a (source, reference) pair into (len(actual_reference) + 2) new examples
    
    Assumes that users always want the empty prefix at the beginning of the generated examples
    (i.e. ask the system for the full hypothesis) and the empty suffix (i.e. ask the system for nothing)
    at the end of the generated examples
    
    By passing None for bos_token or eos_token, user indicates that these tokens have already
    been prepended or appended to the reference
    
    """
    
    start_index = 0
    if bos_token is not None:
        assert reference[0] == bos_token
        start_index = 1
        
    end_index = len(reference) + 1
    if eos_token is not None:
        assert reference[-1] == eos_token
        end_index -= 1
    
    prefixes, suffixes = zip(*[(reference[:i], reference[i:]) for i in range(start_index, end_index)])
    sources = [source for _ in range(end_index - start_index)]
    
    assert len(sources) == len(prefixes) == len(suffixes), 'All components must have the same length'
    
    return zip(sources, prefixes, suffixes)
    

# WORKING: 
# - change the NMTPredictor to IMTPredictor -- change its loading and parsing to convert the dataset
# to imt format on the fly
# - also print some statistics about the dataset for sanity

# This function is only needed at evaluation time

def map_parallel_dataset_to_imt_dataset(source_file, reference_file, bos_token=None, eos_token=None):
    """
    Take a whitespace-delimited parallel dataset, 
    map each (source, target) pair into (target_length) triples of (source, prefix, completion). 
    
    # Working here:
    Note that target_length only applies when BOS and EOS tokens
    are not none

    
    if bos_token is not None, prepend it to target sequence
    
    if eos_token is not None, append it to target sequence
    
    """
    
#     WORKING
# (1) open source file
# (2) split source file into tokens
# (3) map tokens to 
    
#        with codecs.open(input_file) as inp:
#             for i, line in enumerate(inp.read().strip().split('\n')):
    
    
    pass

In [7]:
# WORKING: 
# - IMT evaluation like in NMT with baseline NMT system
# - Simplest baseline: threshold prefixes to a maximum number of words
# - remember that evaluation can take a long time, since we are greatly expanding the number of instances for IMT

In [8]:
# Sample config for test experiments
BASEDIR='/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/BERTHA-TEST_Adam_wmt-multimodal_internal_data_'+\
    'dropout0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/'

exp_config = {
    'src_vocab_size': 20000,
    'trg_vocab_size': 20000,
    'enc_embed': 300,
    'dec_embed': 300,
    'enc_nhids': 800,
    'dec_nhids': 800,
#     'saved_parameters': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout'+\
#     '0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/best_bleu_model_1455464992_BLEU31.61.npz',
    'src_vocab': os.path.join(BASEDIR, 'vocab.en-de.en.pkl'),
    'trg_vocab': os.path.join(BASEDIR, 'vocab.en-de.de.pkl'),
#     'src_data': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout'+\
#     '0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/training_data/train.en.tok.shuf',
#     'trg_data': '/home/chris/projects/neural_mt/archived_models/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout'+\
#     '0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/training_data/train.de.tok.shuf',
    'unk_id':1,
    # Bleu script that will be used (moses multi-perl in this case)
    'bleu_script': '/home/chris/projects/neural_mt/test_data/sample_experiment/tiny_demo_dataset/multi-bleu.perl',
    # Optimization related ----------------------------------------------------
    # Batch size
    'batch_size': 5,
    # This many batches will be read ahead and sorted
    'sort_k_batches': 1,
    # Optimization step rule
    'step_rule': 'AdaDelta',
    # Gradient clipping threshold
    'step_clipping': 1.,
    # Std of weight initialization
    'weight_scale': 0.01,
    'seq_len': 40,
    'finish_after': 100,
    
    # WORKING: add config for evaluation with IMT model -- follow NMT evaluation config

    ##############                                                                                  
    # PREDICTION #                                                                                              
    ##############      
    'prefix_decoding': True,
    'brick_delimiter': '-',
    'beam_size': 10,
    'normalized_bleu': True,

    # Note we _do not_ need the target_prefixes, since these are generated on the fly

    'source_lang': 'en',                                                                                                    
    'target_lang': 'de',                                                                                                                 

    'n_best': 1,                                                                                                                                 

    # path to the moses perl script for tokenization                                                                                    
    'tokenize_script': None,                                                                                                                                
    # path to the moses perl script for detokenization                                                                                              
    'detokenize_script': None,                                                                                                                              

    # The location of the saved parameters of a trained model as .npz                                                                       
    # TODO: model save directory is currently misnamed -- switch to yaml configs with good model names                          
    'saved_parameters': '/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/BERTHA-TEST_Adam_wmt-multimodal_internal_data_'+\
    'dropout0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/'+\
    'best_bleu_model_1455464992_BLEU31.61.npz',
                                                                                                                                                           
                                                                                                                                                        
    # The location of a test set in the source language                                                                                                             
    'test_set': '/media/1tb_drive/multilingual-multimodal/flickr30k/test/test.en.tok',                                   
    #'test_set': '/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/dev.en.tok'                                                        
                                                                                                                            
    # your desired path to the translated output file, or an already-translated file that you just want to evaluate                                           
    'translated_output_file': '/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/test.imt_PROTO-DELETE-ME.30.x.hyps.out',         
    #'translated_output_file': '/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/dev.multimodal-summed.30.x.hyps.out'             
                                                                                                                                                            
    # The location of the gold standard references for the test set (for evaluation mode only)                                                  
    'test_gold_refs': '/media/1tb_drive/multilingual-multimodal/flickr30k/test/test.de.tok', 
    
    # if the config contains this key, meteor will also be computed                                                                                                          │
    'meteor_directory': '/home/chris/programs/meteor-1.5'   
}

In [9]:
from __future__ import print_function
import argparse
import logging
import pprint
import codecs
import re
import os
import time
from subprocess import Popen, PIPE, check_output

from machine_translation import configurations
import logging

import os
import shutil
from collections import Counter
from theano import tensor
from toolz import merge
import numpy
import pickle
from subprocess import Popen, PIPE
import codecs

from blocks.algorithms import (GradientDescent, StepClipping,
                               CompositeRule, Adam, AdaDelta)
from blocks.extensions import FinishAfter, Printing, Timing
from blocks.extensions.monitoring import TrainingDataMonitoring
from blocks.filter import VariableFilter
from blocks.graph import ComputationGraph, apply_noise, apply_dropout
from blocks.initialization import IsotropicGaussian, Orthogonal, Constant
from blocks.main_loop import MainLoop
from blocks.model import Model
from blocks.select import Selector
from blocks.search import BeamSearch
from blocks_extras.extensions.plot import Plot

from machine_translation.checkpoint import CheckpointNMT, LoadNMT
from machine_translation.model import BidirectionalEncoder
from machine_translation.stream import _ensure_special_tokens

# from mmmt.model import InitialContextDecoder

# user can specify which target GRU they want
# from mmmt.model import GRUInitialState, GRUInitialStateWithInitialStateConcatContext, GRUInitialStateWithInitialStateSumContext
from mmmt.sample import BleuValidator, Sampler, SamplingBase

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 4007)


In [10]:
from nn_imt.model import Decoder

# TODO: add SamplingBase to nn_imt

In [ ]:
def load_params_and_get_beam_search(exp_config, brick_delimiter=None):

    encoder = BidirectionalEncoder(
        exp_config['src_vocab_size'], exp_config['enc_embed'], exp_config['enc_nhids'])


    # Create Theano variables
    logger.info('Creating theano variables')
    sampling_input = tensor.lmatrix('source')

    # Get beam search
    logger.info("Building sampling model")
    sampling_representation = encoder.apply(
        sampling_input, tensor.ones(sampling_input.shape))

    # TODO: how to determine the type of decoder that we need?
    # TODO: i.e. how to make prefix decoding configurable when building the decoder?
    # TODO: remove the 'loss_function' param, it's not needed for prediction
    if exp_config.get('prefix_decoding', False):
        decoder = Decoder(
            exp_config['trg_vocab_size'], exp_config['dec_embed'], exp_config['dec_nhids'],
            exp_config['enc_nhids'] * 2, loss_function='min_risk')
        # TODO: should we change the variable name for clarity?
        sampling_prefix = tensor.lmatrix('target')
        generated = decoder.generate(sampling_input, sampling_representation,
                                     target_prefix=sampling_prefix)
    else:
        sampling_prefix = None
        decoder = Decoder(
            exp_config['trg_vocab_size'], exp_config['dec_embed'], exp_config['dec_nhids'],
            exp_config['enc_nhids'] * 2)
        generated = decoder.generate(sampling_input, sampling_representation)

    _, samples = VariableFilter(
        bricks=[decoder.sequence_generator], name="outputs")(
        ComputationGraph(generated[1]))  # generated[1] is next_outputs
    beam_search = BeamSearch(samples=samples)

    # Set the parameters
    logger.info("Creating Model...")
    model = Model(generated)
    logger.info("Loading parameters from model: {}".format(exp_config['saved_parameters']))

    # load the parameter values from an .npz file
    param_values = LoadNMT.load_parameter_values(exp_config['saved_parameters'], brick_delimiter=brick_delimiter)
    LoadNMT.set_model_parameters(model, param_values)

    return beam_search, sampling_input, sampling_prefix

# TODO: the predictor needs modes -- prefix prediction vs prediction from scratch
# TODO: Another option would be to always pass a '0' prefix to keep the theano function interface the same
class IMTPredictor:
    """"Uses a trained NMT model to do prediction"""

    sutils = SamplingBase()

    def __init__(self, exp_config):

        theano_variables = load_params_and_get_beam_search(exp_config, exp_config.get('brick_delimiter', None))
        self.beam_search, self.source_sampling_input, self.target_sampling_input = theano_variables

        self.exp_config = exp_config
        # how many hyps should be output (only used in file prediction mode)
        self.n_best = exp_config.get('n_best', 1)

        self.source_lang = exp_config.get('source_lang', 'en')
        self.target_lang = exp_config.get('target_lang', 'es')

        tokenize_script = exp_config.get('tokenize_script', None)
        detokenize_script = exp_config.get('detokenize_script', None)
        if tokenize_script is not None and detokenize_script is not None:
            self.source_tokenizer_cmd = [tokenize_script, '-l', self.source_lang, '-q', '-', '-no-escape', '1']
            self.target_tokenizer_cmd = [tokenize_script, '-l', self.target_lang, '-q', '-', '-no-escape', '1']
            self.detokenizer_cmd = [detokenize_script, '-l', self.target_lang, '-q', '-']
        else:
            self.source_tokenizer_cmd = None
            self.target_tokenizer_cmd = None
            self.detokenizer_cmd = None

        # this index will get overwritten with the EOS token by _ensure_special_tokens
        # IMPORTANT: the index must be created in the same way it was for training,
        # otherwise the predicted indices will be nonsense
        # Make sure that src_vocab_size and trg_vocab_size are correct in your configuration
        self.src_eos_idx = exp_config['src_vocab_size'] - 1
        self.trg_eos_idx = exp_config['trg_vocab_size'] - 1

        self.unk_idx = exp_config['unk_id']

        # Get vocabularies and inverse indices
        self.src_vocab = _ensure_special_tokens(
            pickle.load(open(exp_config['src_vocab'])), bos_idx=0,
            eos_idx=self.src_eos_idx, unk_idx=self.unk_idx)
        self.src_ivocab = {v: k for k, v in self.src_vocab.items()}
        self.trg_vocab = _ensure_special_tokens(
            pickle.load(open(exp_config['trg_vocab'])), bos_idx=0,
            eos_idx=self.trg_eos_idx, unk_idx=self.unk_idx)
        self.trg_ivocab = {v: k for k, v in self.trg_vocab.items()}

        self.unk_idx = self.unk_idx

    def map_idx_or_unk(self, sentence, index, unknown_token='<UNK>'):
        if type(sentence) is str:
            sentence = sentence.split()
        return [index.get(w, unknown_token) for w in sentence]

# TODO: switch to predict files, like in multimodal_nmt, we need the reference here
# also, because we need to create the input prefixes
# TODO: also (optionally) output the reference suffix file from this function -- 
# this will be useful for external validation,
# otherwise, it will have to be re-created for validation
    def predict_files(self, source_file, reference_file, output_file=None):
        tokenize = self.source_tokenizer_cmd is not None
        detokenize = self.detokenizer_cmd is not None

        if output_file is not None:
            ftrans = codecs.open(output_file, 'wb', encoding='utf8')
        else:
            # cut off the language suffix to make output file name
            output_file = '.'.join(input_file.split('.')[:-1]) + '.trans.out'
            ftrans = codecs.open(output_file, 'wb', encoding='utf8')

        logger.info("Started translation, will output {} translations for each segment"
                    .format(self.n_best))
        total_cost = 0.0

        # TODO: the tokenizer throws an error when the input file is opened with encoding='utf8'
        # why would this happen?
        
        # WORKING:
        with codecs.open(source_file) as srcs:
            with codecs.open(reference_file) as refs:
                # map (source, ref) to [(source, prefix, suffix)]
                source_lines = srcs.read().strip().split('\n')
                ref_lines = refs.read().strip().split('\n')
                assert len(source_lines) == len(ref_lines), 'Source and reference files must be the same length'
                for i, instance in enumerate(zip(source_lines, ref_lines)):
                    logger.info("Translating segment: {}".format(i))

                    # TODO: tokenization is not currently implemented -- assumes whitespace tokenization!!!
                    # Right now, tokenization happens in self.map_idx_or_unk if predict_segment is passed a string
                    source_seq = instance[0].split()
                    target_seq = instance[1].split()
                    
                    # map to triples
                    imt_triples = map_pair_to_imt_triples(source_seq, target_seq)
                    
                    # the first instance contains the empty prefix, the last instance contains the empty suffix
                    # HACK: pop the first item until a fix for empty prefix is implemented
                    imt_triples = imt_triples[1:]
                    
                    for src, prefix, suffix in imt_triples:
                        
                        # TODO: remove suffix from arglist of this function
                        translations, costs = self.predict_segment(src, suffix, target_prefix=prefix,
                                                                   n_best=self.n_best,
                                                                   tokenize=tokenize, detokenize=detokenize)

                        # predict_segment returns a list of hyps, we just take the best one
                        nbest_translations = translations[:self.n_best]
                        nbest_costs = costs[:self.n_best]

                        if self.n_best == 1:
                            ftrans.write((nbest_translations[0] + '\n').decode('utf8'))
                            total_cost += nbest_costs[0]
                        else:
                            # one blank line to separate each nbest list
                            ftrans.write('\n'.join(nbest_translations) + '\n\n')
                            total_cost += sum(nbest_costs)

                    if i != 0 and i % 100 == 0:
                        logger.info("Translated {} lines of test set...".format(i))

        logger.info("Saved translated output to: {}".format(ftrans.name))
        logger.info("Total cost of the test: {}".format(total_cost))
        ftrans.close()

        return output_file

    # TODO: remove suffix from arglist of this function
    def predict_segment(self, segment, suffix, target_prefix=None, n_best=1, tokenize=False, detokenize=False):
        """
        Do prediction for a single segment, which is a list of token idxs

        Parameters
        ----------
        segment: list[int] : a list of int indexes representing the input sequence in the source language
        n_best: int : how many hypotheses to return (must be <= beam_size)
        tokenize: bool : does the source segment need to be tokenized first?
        detokenize: bool : do the output hypotheses need to be detokenized?

        Returns
        -------
        trans_out: str : the best translation according to beam search
        cost: float : the cost of the best translation

        """

        # if there is a prefix, we need to tokenize and preprocess it also
        # TODO: prefix can be a kwarg, but how to transparently create the Decoder?
        # TODO: probably the easiest way is just to always use the PartialSequenceGenerator for
        # TODO: decoders which are for prediction only

        if tokenize:
            source_tokenizer = Popen(self.source_tokenizer_cmd, stdin=PIPE, stdout=PIPE)
            segment, _ = source_tokenizer.communicate(segment)
            if target_prefix is not None:
                target_tokenizer = Popen(self.target_tokenizer_cmd, stdin=PIPE, stdout=PIPE)
                target_prefix, _ = target_tokenizer.communicate(target_prefix)

        segment = self.map_idx_or_unk(segment, self.src_vocab, self.unk_idx)
        segment += [self.src_eos_idx]

        seq = IMTPredictor.sutils._oov_to_unk(
            segment, self.exp_config['src_vocab_size'], self.unk_idx)
        input_ = numpy.tile(seq, (self.exp_config['beam_size'], 1))

        # TODO: HANDLE THE CASE WHERE TARGET PREFIX IS EMPTY
        if target_prefix is not None:
            print('predicting target prefix: {}'.format(target_prefix))
            target_prefix = self.map_idx_or_unk(target_prefix, self.trg_vocab, self.unk_idx)
            prefix_seq = IMTPredictor.sutils._oov_to_unk(
                target_prefix, self.exp_config['trg_vocab_size'], self.unk_idx)
            
            # TODO: remove suffix from this function -- put it outside
            target_suffix = self.map_idx_or_unk(suffix, self.trg_vocab, self.unk_idx)
            suffix_seq = IMTPredictor.sutils._oov_to_unk(
                target_suffix, self.exp_config['trg_vocab_size'], self.unk_idx)
            
            
            prefix_input_ = numpy.tile(prefix_seq, (self.exp_config['beam_size'], 1))
            # draw sample, checking to ensure we don't get an empty string back
            trans, costs = \
                self.beam_search.search(
                    input_values={self.source_sampling_input: input_,
                                  self.target_sampling_input: prefix_input_},
                    max_length=3*len(seq), eol_symbol=self.trg_eos_idx,
                    ignore_first_eol=True)
            # import ipdb;ipdb.set_trace()

        else:
            # draw sample, checking to ensure we don't get an empty string back
            trans, costs = \
                self.beam_search.search(
                    input_values={self.sampling_input: input_},
                    max_length=3*len(seq), eol_symbol=self.trg_eos_idx,
                    ignore_first_eol=True)

        # normalize costs according to the sequence lengths
        if self.exp_config['normalized_bleu']:
            lengths = numpy.array([len(s) for s in trans])
            costs = costs / lengths

        best_n_hyps = []
        best_n_costs = []
        best_n_idxs = numpy.argsort(costs)[:n_best]
        for j, idx in enumerate(best_n_idxs):
            try:
                trans_out_idxs = trans[idx]
                cost = costs[idx]

                # convert idx to words
                # `line` is a tuple with one item
                try:
                    assert trans_out_idxs[-1] == self.trg_eos_idx, 'Target hypothesis should end with the EOS symbol'
                    trans_out_idxs = trans_out_idxs[:-1]
                    src_in = IMTPredictor.sutils._idx_to_word(segment, self.src_ivocab)
                    trans_out = IMTPredictor.sutils._idx_to_word(trans_out_idxs, self.trg_ivocab)
                except AssertionError as e:
                    src_in = IMTPredictor.sutils._idx_to_word(segment, self.src_ivocab)
                    trans_out = IMTPredictor.sutils._idx_to_word(trans_out, self.trg_ivocab)
                    logger.error("ERROR: {} does not end with the EOS symbol".format(trans_out))
                    logger.error("I'm continuing anyway...")
            except ValueError:
                logger.info("Can NOT find a translation for line: {}".format(src_in))
                trans_out = '<UNK>'
                cost = 0.
            
            # compute score for trans_out_idxs and reference suffix
            # TODO: move this outside of the predict_segment function!
            imt_f1_score = imt_f1(trans_out_idxs, suffix_seq)

            if detokenize:
                detokenizer = Popen(self.detokenizer_cmd, stdin=PIPE, stdout=PIPE)
                trans_out, _ = detokenizer.communicate(trans_out)
                # strip off the eol symbol
                trans_out = trans_out.strip()

            # TODO: remove this quick hack
            trans_out = trans_out.replace('<UNK>', 'UNK')

            logger.info("Source: {}".format(src_in))
            logger.info("Target Hypothesis: {}".format(trans_out))
            logger.info("Target Reference: {}".format(suffix))
            logger.info("IMT F1 score: {}".format(imt_f1_score))

            best_n_hyps.append(trans_out)
            best_n_costs.append(cost)

        return best_n_hyps, best_n_costs

In [ ]:
# create the control function which will run evaluation
config_obj = exp_config


# elif mode == 'evaluate':
#     logger.info("Started Evaluation: ")
#     val_start_time = time.time()

# translate if necessary, write output file, call external evaluation tools and show output
translated_output_file = config_obj.get('translated_output_file', None)
#     if translated_output_file is not None and os.path.isfile(translated_output_file):
#             logger.info('{} already exists, so I\'m evaluating the BLEU score of this file with respect to the ' +
#                         'reference that you provided: {}'.format(translated_output_file,
#                                                                  config_obj['test_gold_refs']))
#     else:


predictor = IMTPredictor(config_obj)
logger.info('Translating: {}'.format(config_obj['test_set']))
translated_output_file = predictor.predict_files(config_obj['test_set'],
                                                 config_obj['test_gold_refs'],
                                                 translated_output_file)
logger.info('Translated: {}, output was written to: {}'.format(config_obj['test_set'],
                                                               translated_output_file))

INFO:__main__:Creating theano variables
INFO:__main__:Building sampling model

Your function uses a non-shared variable other than those given by scan explicitly. That can significantly slow down `tensor.grad` call. Did you forget to declare it in `contexts`?
INFO:__main__:Creating Model...
INFO:__main__:Loading parameters from model: /media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/best_bleu_model_1455464992_BLEU31.61.npz
INFO:__main__:Translating: /media/1tb_drive/multilingual-multimodal/flickr30k/test/test.en.tok
INFO:__main__:Started translation, will output 1 translations for each segment
INFO:__main__:Translating segment: 0


{'target_prefix': decoder_generate_target_prefix}
target_prefix: sequencegenerator_initial_states_target_prefix
predicting target prefix: ['Ein']

/home/chris/anaconda2/lib/python2.7/site-packages/blocks/search.py:113: UserWarning: theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 3 is not part of the computational graph needed to compute the outputs: sequencegenerator_generate_outputs.
To make this warning into an error, you can pass the parameter on_unused_input='raise' to theano.function. To disable it completely, use on_unused_input='ignore'.
  self.contexts + self.input_states + next_outputs, next_states)
/home/chris/anaconda2/lib/python2.7/site-packages/blocks/search.py:323: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  for output, mask in equizip(outputs, masks)]
INFO:__main__:Source: A man dressed in green snowboarding on a bench . </S>
INFO:__main__:Target Hypothesis: Mann in grün fährt auf einer Bank .
INFO:__main__:Target Reference: ['gr\xc3\xbcn', 'gekleideter', 'Mann', 'f


predicting target prefix: ['Ein', 'gr\xc3\xbcn']

INFO:__main__:Source: A man dressed in green snowboarding on a bench . </S>
INFO:__main__:Target Hypothesis: gekleideter Mann fährt mit seinem Snowboard auf einer Bank .
INFO:__main__:Target Reference: ['gekleideter', 'Mann', 'f\xc3\xa4hrt', 'Snowboard', 'auf', 'einer', 'Bank', '.']
INFO:__main__:IMT F1 score: 0.333333333333


...


predicting target prefix: ['Ein', 'schwarzer', 'Mann', 'in', 'einer', 'Kochjacke']

INFO:__main__:Source: A black man in a chef 's coat is on the phone while two other men in aprons laugh . </S>
INFO:__main__:Target Hypothesis: , zwei andere Männer mit Schürzen lachen .
INFO:__main__:Target Reference: ['telefoniert', ',', 'w\xc3\xa4hrend', 'zwei', 'andere', 'M\xc3\xa4nner', 'mit', 'Sch\xc3\xbcrzen', 'lachen', '.']
INFO:__main__:IMT F1 score: 0.0



predicting target prefix: ['Ein', 'schwarzer', 'Mann', 'in', 'einer', 'Kochjacke', 'telefoniert']

INFO:__main__:Source: A black man in a chef 's coat is on the phone while two other men in aprons laugh . </S>
INFO:__main__:Target Hypothesis: , während zwei andere Männer im Schürzen lachen .
INFO:__main__:Target Reference: [',', 'w\xc3\xa4hrend', 'zwei', 'andere', 'M\xc3\xa4nner', 'mit', 'Sch\xc3\xbcrzen', 'lachen', '.']
INFO:__main__:IMT F1 score: 0.555555555556



predicting target prefix: ['Ein', 'schwarzer', 'Mann', 'in', 'einer', 'Kochjacke', 'telefoniert', ',']

In [ ]:
# TESTS

# with BOS and EOS
s_test1 = ['<S>', 'He', 'is', 'a', 'nice', 'boy', '.', '</S>']
t_test1 = ['<S>', 'Er', 'ist', 'ein', 'netter', 'Junge', '.', '</S>']

# without BOS and EOS
s_test2 = ['He', 'is', 'a', 'nice', 'boy', '.']
t_test2 = ['Er', 'ist', 'ein', 'netter', 'Junge', '.']

# map_pair_to_imt_triples(s_test1, t_test1, bos_token='<S>', eos_token='</S>')
map_pair_to_imt_triples(s_test2, t_test2)

In [ ]:
# coding: utf-8

import os
import codecs
import subprocess
from pprint import pprint
from subprocess import Popen, PIPE, STDOUT

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

import numpy
import codecs
import tempfile
import cPickle
import copy
from collections import OrderedDict
import itertools
from theano import tensor

from fuel.datasets import Dataset
from fuel.datasets import TextFile
from fuel.schemes import ConstantScheme
from fuel.streams import DataStream
from fuel.transformers import (
    Merge, Batch, Filter, Padding, SortMapping, Unpack, Mapping)
from fuel.transformers import Transformer

from blocks.algorithms import (GradientDescent, StepClipping,
                               CompositeRule, Adam, AdaDelta)
from blocks.extensions import FinishAfter, Printing, Timing
from blocks.extensions.monitoring import TrainingDataMonitoring
from blocks.filter import VariableFilter
from blocks.graph import ComputationGraph, apply_noise, apply_dropout
from blocks.initialization import IsotropicGaussian, Orthogonal, Constant
from blocks.main_loop import MainLoop
from blocks.model import Model
from blocks.select import Selector
from blocks.search import BeamSearch
from blocks_extras.extensions.plot import Plot

from machine_translation.checkpoint import CheckpointNMT, LoadNMT
from machine_translation.model import BidirectionalEncoder, Decoder
from machine_translation.sampling import BleuValidator, Sampler, SamplingBase
from machine_translation.stream import (get_tr_stream, get_dev_stream,
                                        _ensure_special_tokens, MTSampleStreamTransformer,
                                        get_textfile_stream, _too_long, _length, PaddingWithEOS,
                                        _oov_to_unk)


from machine_translation.evaluation import sentence_level_bleu

try:
    from blocks_extras.extensions.plot import Plot
    BOKEH_AVAILABLE = True
except ImportError:
    BOKEH_AVAILABLE = False


# build the training and sampling graphs for minimum risk training
# Intialize the MTSampleStreamTransformer with the sampling function

# load a model that's already trained, and start tuning it with minimum-risk
# mock-up training using the blocks main loop

# TODO: Integrate configuration so min-risk training is a single line in the config file
# TODO: this requires handling both the data stream and the Model cost function

# create the graph which can sample from our model
# Note that we must sample instead of getting the 1-best or N-best, because we need the randomness to make the expected
# BLEU score make sense

BASEDIR = '/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/' +\ 
              'BERTHA-TEST_Adam_wmt-multimodal_internal_data_dropout'+\
              '0.3_ff_noiseFalse_search_model_en2es_vocab20000_emb300_rec800_batch15/'
#best_bleu_model_1455464992_BLEU31.61.npz

exp_config = {
    'src_vocab_size': 20000,
    'trg_vocab_size': 20000,
    'enc_embed': 300,
    'dec_embed': 300,
    'enc_nhids': 800,
    'dec_nhids': 800,
    'saved_parameters': '/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/' +\
        'BERTHA-TEST_wmt-multimodal_internal_data_dropout0.3_ff_noiseFalse_search_model_en2es_vocab20000' +\ 
        '_emb300_rec800_batch15/best_bleu_model_1455410311_BLEU30.38.npz',
    'src_vocab': os.path.join(BASEDIR, 'vocab.en-de.en.pkl'),
    'trg_vocab': os.path.join(BASEDIR, 'vocab.en-de.de.pkl'),
    'src_data': os.path.join(BASEDIR, 'training_data/train.en.tok.shuf'),
    'trg_data': os.path.join(BASEDIR, 'training_data/train.de.tok.shuf'),
    'unk_id':1,
    # Bleu script that will be used (moses multi-perl in this case)
    'bleu_script': os.path.join(os.path.dirname(os.path.realpath(__file__)),
                                '../test_data/sample_experiment/tiny_demo_dataset/multi-bleu.perl'),

    # Optimization related ----------------------------------------------------
    # Batch size
    'batch_size': 8,
    # This many batches will be read ahead and sorted
    'sort_k_batches': 2,
    # Optimization step rule
    'step_rule': 'AdaDelta',
    # Gradient clipping threshold
    'step_clipping': 1.,
    # Std of weight initialization
    'weight_scale': 0.01,
    'seq_len': 40,
    # Beam-size
    'beam_size': 10,

    # Maximum number of updates
    'finish_after': 1000000,

    # Reload model from files if exist
    'reload': False,

    # Save model after this many updates
    'save_freq': 500,

    # Show samples from model after this many updates
    'sampling_freq': 1000,

    # Show this many samples at each sampling
    'hook_samples': 5,

    # Validate bleu after this many updates
    'bleu_val_freq': 10,
    # Normalize cost according to sequence length after beam-search
    'normalized_bleu': True,
    
    'saveto': '/media/1tb_drive/test_min_risk_model_save',
    'model_save_directory': 'test_min_risk_model_save',
    # Validation set source file
    'val_set': '/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/dev.en.tok',

    # Validation set gold file
    'val_set_grndtruth': '/media/1tb_drive/multilingual-multimodal/flickr30k/train/processed/dev.de.tok',

    # Print validation output to file
    'output_val_set': True,

    # Validation output file
    'val_set_out': '/media/1tb_drive/test_min_risk_model_save/validation_out.txt',
    'val_burn_in': 0,

    # NEW PARAM FOR MIN RISK TRAINING
    'n_samples': 100

}

# this returns the function for sampling from the model
def get_sampling_model_and_input(exp_config):
    # Create Theano variables
    encoder = BidirectionalEncoder(
        exp_config['src_vocab_size'], exp_config['enc_embed'], exp_config['enc_nhids'])

#   TODO: decoder with special generate function which can take a prefix as input
    decoder = Decoder(
        exp_config['trg_vocab_size'], exp_config['dec_embed'], exp_config['dec_nhids'],
        exp_config['enc_nhids'] * 2, loss_function='min_risk')

    # Create Theano variables
    logger.info('Creating theano variables')
    sampling_input = tensor.lmatrix('source')

    # TODO: beam search which can take (source, prefix) as input
    # Get beam search
    logger.info("Building sampling model")
    sampling_representation = encoder.apply(
        sampling_input, tensor.ones(sampling_input.shape))
    generated = decoder.generate(sampling_input, sampling_representation)

#     _, samples = VariableFilter(
#         bricks=[decoder.sequence_generator], name="outputs")(
#                  ComputationGraph(generated[1]))  # generated[1] is next_outputs
#     beam_search = BeamSearch(samples=samples)

    # build the model that will let us get a theano function from the sampling graph
    logger.info("Creating Sampling Model...")
    sampling_model = Model(generated)

    return sampling_model, sampling_input, encoder, decoder

sample_model, theano_sampling_input, train_encoder, train_decoder = get_sampling_model_and_input(exp_config)

In [ ]:
# test that we can pull samples from the model
trg_vocab = cPickle.load(open(exp_config['trg_vocab']))
trg_vocab_size = exp_config['trg_vocab_size'] - 1
src_vocab = cPickle.load(open(exp_config['src_vocab']))
src_vocab_size = exp_config['src_vocab_size'] - 1

src_vocab = _ensure_special_tokens(src_vocab, bos_idx=0,
                                   eos_idx=src_vocab_size, unk_idx=exp_config['unk_id'])
trg_vocab = _ensure_special_tokens(trg_vocab, bos_idx=0,
                                   eos_idx=trg_vocab_size, unk_idx=exp_config['unk_id'])

theano_sample_func = sample_model.get_theano_function()

# note: we close over the sampling func and the trg_vocab to standardize the interface
# TODO: actually this should be a callable class with params (sampling_func, trg_vocab)
# TODO: we may be able to make this function faster by passing multiple sources for sampling at the same damn time
# TODO: or by avoiding the for loop somehow
def sampling_func(source_seq, num_samples=1):

    def _get_true_length(seqs, vocab):
        try:
            lens = []
            for r in seqs.tolist():
                lens.append(r.index(vocab['</S>']) + 1)
            return lens
        except ValueError:
            return [seqs.shape[1] for _ in range(seqs.shape[0])]

    # samples = []
    # for _ in range(num_samples):
        # outputs of self.sampling_fn = outputs of sequence_generator.generate: next_states + [next_outputs] +
        #                 list(next_glimpses.values()) + [next_costs])
        # _1, outputs, _2, _3, costs = theano_sample_func(source_seq[None, :])
        # if we are generating a single sample, the length of the output will be len(source_seq)*2
        # see decoder.generate
        # the output is a [seq_len, 1] array
        # outputs = outputs.reshape(outputs.shape[0])
        # outputs = outputs[:_get_true_length(outputs, trg_vocab)]
        # samples.append(outputs)

    inputs = numpy.tile(source_seq[None, :], (num_samples, 1))
    # the output is [seq_len, batch]
    _1, outputs, _2, _3, costs = theano_sample_func(inputs)
    outputs = outputs.T

    # TODO: this step could be avoided by computing the samples mask in a different way
    lens = _get_true_length(outputs, trg_vocab)
    samples = [s[:l] for s,l in zip(outputs.tolist(), lens)]

    return samples

In [ ]:
src_stream = get_textfile_stream(source_file=exp_config['src_data'], src_vocab=exp_config['src_vocab'],
                                         src_vocab_size=exp_config['src_vocab_size'])

# test_source_stream.sources = ('sources',)
trg_stream = get_textfile_stream(source_file=exp_config['trg_data'], src_vocab=exp_config['trg_vocab'],
                                         src_vocab_size=exp_config['trg_vocab_size'])

# Merge them to get a source, target pair
training_stream = Merge([src_stream,
                         trg_stream],
                         ('source', 'target'))

# Filter sequences that are too long
training_stream = Filter(training_stream,
                         predicate=_too_long(seq_len=exp_config['seq_len']))

# sampling_transformer = MTSampleStreamTransformer(sampling_func, fake_score, num_samples=5)
sampling_transformer = MTSampleStreamTransformer(sampling_func, sentence_level_bleu, num_samples=exp_config['n_samples'])

training_stream = Mapping(training_stream, sampling_transformer, add_sources=('samples', 'scores'))

# TODO: this method connects the data stream with the decoder cost function, is there a better way to decouple?
class FlattenSamples(Transformer):
    """Flatten one dimension from the samples, the reshaping for cost computation
    is done inside the cost function

    Parameters
    ----------
    data_stream : :class:`AbstractDataStream` instance
        The data stream to wrap

    """
    def __init__(self, data_stream, **kwargs):
        if data_stream.produces_examples:
            raise ValueError('the wrapped data stream must produce batches of '
                             'examples, not examples')
        super(FlattenSamples, self).__init__(
            data_stream, produces_examples=False, **kwargs)

#         if mask_dtype is None:
#             self.mask_dtype = config.floatX
#         else:
#             self.mask_dtype = mask_dtype

    @property
    def sources(self):
        return self.data_stream.sources
#         sources = []
#         for source in self.data_stream.sources:
#             sources.append(source)
#             if source in self.mask_sources:
#                 sources.append(source + '_mask')
#         return tuple(sources)

    def transform_batch(self, batch):
        batch_with_flattened_samples = []
        for i, (source, source_batch) in enumerate(
                zip(self.data_stream.sources, batch)):
#             if source not in self.mask_sources:
#                 batch_with_masks.append(source_batch)
#                 continue
            if source == 'samples':
                flattened_samples = []
                for ins in source_batch:
                    for sample in ins:
                        flattened_samples.append(sample)
                batch_with_flattened_samples.append(flattened_samples)
            else:
                batch_with_flattened_samples.append(source_batch)

        return tuple(batch_with_flattened_samples)

# TODO: modify this transformer to split a (source, target) stream into (source, prefix, completion) triples
class CopySourceNTimes(Transformer):
    """Duplicate the source N times to match the number of samples

    We need this transformer because the attention model expects one source sequence for each
    target sequence, but in the sampling case there are effectively (instances*sample_size) target sequences

    Parameters
    ----------
    data_stream : :class:`AbstractDataStream` instance
        The data stream to wrap
    n_samples : int -- the number of samples that were generated for each source sequence

    """
    def __init__(self, data_stream, n_samples=5, **kwargs):
        if data_stream.produces_examples:
            raise ValueError('the wrapped data stream must produce batches of '
                             'examples, not examples')
        self.n_samples = n_samples

        super(CopySourceNTimes, self).__init__(
            data_stream, produces_examples=False, **kwargs)


    @property
    def sources(self):
        return self.data_stream.sources

    def transform_batch(self, batch):
        batch_with_expanded_source = []
        for i, (source, source_batch) in enumerate(
                zip(self.data_stream.sources, batch)):
            if source == 'source':
#                 copy each source seqoyuence self.n_samples times, but keep the tensor 2d

                expanded_source = []
                for ins in source_batch:
                    expanded_source.extend([ins for _ in range(self.n_samples)])

                batch_with_expanded_source.append(expanded_source)
            else:
                batch_with_expanded_source.append(source_batch)

        return tuple(batch_with_expanded_source)



# Replace out of vocabulary tokens with unk token
# training_stream = Mapping(training_stream,
#                  _oov_to_unk(src_vocab_size=exp_config['src_vocab_size'],
#                              trg_vocab_size=exp_config['trg_vocab_size'],
#                              unk_id=exp_config['unk_id']))

# Build a batched version of stream to read k batches ahead
training_stream = Batch(training_stream,
               iteration_scheme=ConstantScheme(
                   exp_config['batch_size']*exp_config['sort_k_batches']))

# Sort all samples in the read-ahead batch
training_stream = Mapping(training_stream, SortMapping(_length))

# Convert it into a stream again
training_stream = Unpack(training_stream)

# Construct batches from the stream with specified batch size
training_stream = Batch(
    training_stream, iteration_scheme=ConstantScheme(exp_config['batch_size']))

# Pad sequences that are short
# IDEA: add a transformer which flattens the target samples before we add the mask
flat_sample_stream = FlattenSamples(training_stream)

expanded_source_stream = CopySourceNTimes(flat_sample_stream, n_samples=exp_config['n_samples'])

# TODO: some sources can be excluded from the padding Op, but since blocks matches sources with input variable
# TODO: names, it's not critical
masked_stream = PaddingWithEOS(
    expanded_source_stream, [exp_config['src_vocab_size'] - 1, exp_config['trg_vocab_size'] - 1])


def create_model(encoder, decoder):

    # Create Theano variables
    logger.info('Creating theano variables')
    source_sentence = tensor.lmatrix('source')
    source_sentence_mask = tensor.matrix('source_mask')

#     target_samples = tensor.tensor3('samples').astype('int64')
#     target_samples_mask = tensor.tensor3('target_samples_mask').astype('int64')
    samples = tensor.lmatrix('samples')
    samples_mask = tensor.matrix('samples_mask')

    # scores is (batch, samples)
    scores = tensor.matrix('scores')
    # We don't need a scores mask because there should be the same number of scores for each instance
    # num samples is a hyperparameter of the model

    # the name is important to make sure pre-trained params get loaded correctly
#     decoder.name = 'decoder'

    # This is the part that is different for the MinimumRiskSequenceGenerator
    cost = decoder.expected_cost(
        encoder.apply(source_sentence, source_sentence_mask),
        source_sentence_mask, samples, samples_mask, scores)


    return cost


def main(model, cost, config, tr_stream, dev_stream, use_bokeh=False):

    # Set the parameters from a trained models (.npz file)
    logger.info("Loading parameters from model: {}".format(exp_config['saved_parameters']))
    # Note the brick delimeter='-' is here for legacy reasons because blocks changed the serialization API
    param_values = LoadNMT.load_parameter_values(config['saved_parameters'], brick_delimiter='-')
    LoadNMT.set_model_parameters(model, param_values)

    logger.info('Creating computational graph')
    cg = ComputationGraph(cost)

    # create the training directory, and copy this config there if directory doesn't exist
#     if not os.path.isdir(config['saveto']):
#         os.makedirs(config['saveto'])
#         shutil.copy(config['config_file'], config['saveto'])

    # Set extensions
    logger.info("Initializing extensions")
    extensions = [
        FinishAfter(after_n_batches=config['finish_after']),
        TrainingDataMonitoring([cost], after_batch=True),
        Printing(after_batch=True),
         CheckpointNMT(config['saveto'],
                       every_n_batches=config['save_freq'])
    ]


    # Set up beam search and sampling computation graphs if necessary

    if config['hook_samples'] >= 1 or config['bleu_script'] is not None:
        logger.info("Building sampling model")
        sampling_representation = train_encoder.apply(
            theano_sampling_input, tensor.ones(theano_sampling_input.shape))
        # TODO: the generated output actually contains several more values, ipdb to see what they are
        generated = train_decoder.generate(theano_sampling_input, sampling_representation)
        search_model = Model(generated)
        _, samples = VariableFilter(
            bricks=[train_decoder.sequence_generator], name="outputs")(
                ComputationGraph(generated[1]))  # generated[1] is next_outputs

    # Add sampling
    if config['hook_samples'] >= 1:
        logger.info("Building sampler")
        extensions.append(
            Sampler(model=search_model, data_stream=tr_stream,
                    hook_samples=config['hook_samples'],
                    every_n_batches=config['sampling_freq'],
                    src_vocab_size=config['src_vocab_size']))

    # Add early stopping based on bleu
    if config['bleu_script'] is not None:
        logger.info("Building bleu validator")
        extensions.append(
            BleuValidator(theano_sampling_input, samples=samples, config=config,
                          model=search_model, data_stream=dev_stream,
                          normalize=config['normalized_bleu'],
                          every_n_batches=config['bleu_val_freq']))

    # Reload model if necessary
    if config['reload']:
        extensions.append(LoadNMT(config['saveto']))

    # Plot cost in bokeh if necessary
    if use_bokeh and BOKEH_AVAILABLE:
        extensions.append(
            Plot(config['model_save_directory'], channels=[['decoder_cost_cost'], ['validation_set_bleu_score']],
                 every_n_batches=10))

    # Set up training algorithm
    logger.info("Initializing training algorithm")
    # if there is dropout or random noise, we need to use the output of the modified graph
#     if config['dropout'] < 1.0 or config['weight_noise_ff'] > 0.0:
#         algorithm = GradientDescent(
#             cost=cg.outputs[0], parameters=cg.parameters,
#             step_rule=CompositeRule([StepClipping(config['step_clipping']),
#                                      eval(config['step_rule'])()])
#         )
#     else:
#         algorithm = GradientDescent(
#             cost=cost, parameters=cg.parameters,
#             step_rule=CompositeRule([StepClipping(config['step_clipping']),
#                                      eval(config['step_rule'])()])
#         )

    algorithm = GradientDescent(
        cost=cost, parameters=cg.parameters,
        step_rule=CompositeRule([StepClipping(config['step_clipping']),
                                 eval(config['step_rule'])()],
                               ),
        on_unused_sources='warn'
    )

    # enrich the logged information
    extensions.append(
        Timing(every_n_batches=100)
    )

    # Initialize main loop
    logger.info("Initializing main loop")
    main_loop = MainLoop(
        model=model,
        algorithm=algorithm,
        data_stream=tr_stream,
        extensions=extensions
    )

    # Train!
    main_loop.run()


training_cost = create_model(train_encoder, train_decoder)


# Set up training model
logger.info("Building model")
train_model = Model(training_cost)


# test_iter = masked_stream.get_epoch_iterator()

# source, source_mask, target, target_mask, samples, samples_mask, scores, scores_mask = test_iter.next()

# train_model.inputs

# test_func = train_model.get_theano_function()

# scores = scores.astype('float32')
# out = test_func(scores, samples_mask, source_mask, source, samples)


# numpy.exp(out)
# out[0].shape
# out
# scores
# src_ivocab = {v:k for k,v in src_vocab.items()}

dev_stream = get_dev_stream(**exp_config)

main(train_model, training_cost, exp_config, masked_stream, dev_stream=dev_stream, use_bokeh=True)